In [1]:
using DataFrames, Optim, Revise, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__);

In [3]:
using BLPmodule; const m = BLPmodule;

In [4]:
df = deserialize("jls/df.jls");
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas = deserialize("jls/deltas.jls");

---

In [5]:
# linear characteristics (staffing variables):
X1 = vcat(transpose.([ec.firms[i].X for i in eachindex(ec.firms)])...)
Z = X1;

In [6]:
initial_θ2 = [2.,3.]
initial_δ = zeros(length(ec.firms), 1);

In [7]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
m.gmm_lm([10.,10.], ec, pars, X1)

θ2 = [10.0, 10.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)


151.94046796610795

In [8]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
m.gmm_grad([1.,1.], ec, pars, X1)

2×1 Matrix{Float64}:
 69.0354433280927
  0.010703836079696348

In [9]:
function g!(storage::Array, θ2::Array)
    storage[:] = m.gmm_grad(θ2, ec, pars, X1)
end

g! (generic function with 1 method)

In [14]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), g!, initial_θ2, LBFGS(), Optim.Options(f_calls_limit =12, x_tol = 1e-6))

θ2 = [2.0, 3.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.6881755068558588; -0.24005032429700912]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [-81.04395216001694, 2.711844816244656]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-1.192982213858955; -0.48267550590964925]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [-39.52197608000847, 2.855922408122328]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-1.2248115956469965; -0.3742043045924084]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [-18.760988040004236, 2.9279612040611642]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.9653393841366102; -0.22205202342841868]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [-8.380494020002118, 2.963980602030582]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.8599924974447836; -0.20881771880197375]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


...

θ2 = [1.999999999999999, 3.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.8163943789884929; -0.21379057308568217]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [1.9999999999999993, 3.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.8163946190706246; -0.21378576837035745]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [1.9999999999999998, 3.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.8163948417134645; -0.2137813126644581]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


θ2 = [2.0, 3.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:10)
θ1 = [-0.816395048191596; -0.21377718045991667]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:17)


 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     4.054825e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 1.0e-06
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 1.02e+02 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.52e-01 ≰ 0.0e+00
    |g(x)|                 = 4.52e+01 ≰ 1.0e-08

 * Work counters
    Seconds run:   36  (vs limit Inf)
    Iterations:    1
    f(x) calls:    61
    ∇f(x) calls:   61


In [15]:
println(Optim.minimizer(res))
println(Optim.initial_state(res))

[2.0, 3.0]
[2.0, 3.0]


---